In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pinneit import Circ, Rect, plot_geo, exclude_points_in_region, plot_prediction

In [ ]:
Ω_bc = {"material":"mat1","mat_val":0, "bc":"outer"} # mat1 -> background
Ω = Rect(-1, -1, 2, 2, BC = Ω_bc)
Γ_bc = {"material":"mat2","mat_val":100, "bc":"bc_circle1"} # mat2 -> electrode
Γ = Circ(0.5, 0.5, 0.1, BC = Γ_bc)
Γ_bc = {"material":"mat3","mat_val":-100, "bc":"bc_circle2"} # mat2 -> electrode
Γ2 = Circ(-0.5, -0.5, 0.1, BC = Γ_bc)

In [ ]:
plot_geo([Ω, Γ,Γ2])

In [ ]:
from netgen.geom2d import CSG2d, Circle, Rectangle
from ngsolve.webgui import Draw
from ngsolve import Mesh, VOL, H1, GridFunction,BilinearForm

def create_geo_component(geometry):
    if geometry.geo_type == "Rect":
        domain = Rectangle(pmin=(geometry.x_min,geometry.y_min), pmax=(geometry.x_max,geometry.y_max), mat=geometry.BC["material"], bc=geometry.BC["bc"] )
    elif geometry.geo_type == "Circ":
        domain = Circle( center=(geometry.x,geometry.y), radius=geometry.r, mat=geometry.BC["material"], bc=geometry.BC["bc"] )
    return domain


def wrap_to_mesh(main, interior_s, refinement = 0.05, draw = False):
    
    if isinstance(interior_s, list):
        print(f"Got a list of {len(interior_s)} geometries.")
    else:
        interior_s = [interior_s]
        
    geo = CSG2d()
    
    regionCFf_dict = {main.BC["material"] : main.BC["mat_val"]}
    background = create_geo_component(main)

    for ints in interior_s:
        regionCFf_dict[ints.BC["material"]] = ints.BC["mat_val"]
        sub_domain = create_geo_component(ints)
        background = background - sub_domain # "-" is subtsraction, "*" is union
    geo.Add(background)

    mesh = geo.GenerateMesh(maxh=refinement)
    mesh = Mesh(mesh)
    # curve the mesh elements for geometry approximation of given order
    mesh.Curve(3)
    print("boundaries:",mesh.GetBoundaries())
    #cf = mesh.RegionCF(VOL, regionCFf_dict)
    if draw:
        # Draw(cf, mesh)
        Draw (mesh)
    return mesh

In [ ]:
mesh = wrap_to_mesh(Ω,[Γ,Γ2])

In [ ]:
Draw(mesh)

## Example

In [ ]:
from ngsolve import unit_square, x

In [ ]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))#maxh=0.25))

In [ ]:
mesh.nv, mesh.ne

In [ ]:
fes = H1(mesh, order=2, dirichlet="top|left|right")
fes.ndof  # number of unknowns in this space

In [ ]:
u = fes.TrialFunction()  # symbolic object
v = fes.TestFunction()   # symbolic object

# u, v = fes.TnT()

gfu = GridFunction(fes)  # solution 


In [ ]:
a = BilinearForm(fes, symmetric=True)
a += SymbolicBFI(grad(u)*grad(v))
a.Assemble()

f = LinearForm(fes)
f += SymbolicLFI(x*v)
f.Assemble()

In [ ]:
#gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs(), inverse='umfpack')*f.vec
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs(), inverse='sparsecholesky')*f.vec
#gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Draw(gfu)

## 2D

In [2]:
from netgen.geom2d import CSG2d, Circle, Rectangle
from ngsolve import Mesh
from ngsolve.webgui import Draw

In [ ]:
geo = CSG2d()
#el1 = Circle(center=(0.5,0.5), radius=0.1, bc = "bc_el1", mat = "el_mat")
el1 = Rectangle(pmin=(-1.5,-0.5), pmax=(-1,0.5), bc = "bc_el1", mat = "el_mat")
el2 = Rectangle(pmin=(1.5,-0.5), pmax=(1.5,0.5), bc = "bc_el2", mat = "el_mat")
domain = Rectangle(pmin=(-3,-2), pmax=(3,2), bc = "default", mat = "air") - el1 - el2
geo.Add(domain)
geo.Add(el1)
mesh = Mesh(geo.GenerateMesh(maxh=0.2))

In [ ]:
Draw(mesh)

In [15]:
print(mesh.GetBoundaries())
print(mesh.GetMaterials())

('default', 'default', 'default', 'default', 'bc_el1', 'bc_el1', 'bc_el1', 'bc_el1')
('air', 'el_mat')


In [16]:
from collections import defaultdict
from ngsolve import CoefficientFunction

In [17]:
diel_perm = defaultdict(lambda: -1) #-1 ist default
# all diel_perm["unknwn"] = -1
diel_perm["air"] = 1.0
diel_perm_cf = CoefficientFunction([diel_perm[mat] for mat in mesh.GetMaterials()])

voltage = 4
potential = defaultdict(lambda : 0)
potential["bc_el1"] = voltage
potential_cf = CoefficientFunction([potential[bnd] for bnd in mesh.GetBoundaries()])
potential_str="bc_el1" #|bc_el2 ,...

In [18]:
Draw(potential_cf,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [19]:
dirichlet_str="air|bc_el1"

In [20]:
from ngsolve import H1, BilinearForm,grad,dx, GridFunction, Grad

In [21]:
fes = H1(mesh, order = 2, dirichlet = dirichlet_str)
ut,vt = fes.TnT()
a = BilinearForm(fes)

a+= diel_perm_cf * grad(ut)*grad(vt)*dx
a.Assemble()
u = GridFunction(fes, name="Potential")
u.Set(potential_cf, definedon=mesh.Boundaries(potential_str))
f = u.vec.CreateVector()
f.data = a.mat*u.vec
u.vec.data -= a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky") *f

In [22]:
Draw(u)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

## 3d plate capacitor

In [ ]:
from netgen.occ import *
#from ngsolve import *
from ngsolve.webgui import Draw
# import netgen.gui
from ngsolve import Mesh

In [ ]:
# gen mesh
el1 = Box(Pnt(-0.5,-0.5,-0.6),Pnt(0.5,0.5,-0.5)).bc("bc_el1").mat("egal")
el2 = Box(Pnt(-0.5,-0.5,0.5),Pnt(0.5,0.5,0.6)).bc("bc_el2").mat("egal")
air = Box(Pnt(-0.5,-0.5,-0.6), Pnt(0.5,0.5,0.6)) - el1 - el2

el1.mat("unknwn")
el2.mat("unknwn")
air.mat("air")
geo = OCCGeometry(Glue([air, el1, el2]))
mesh = Mesh(geo.GenerateMesh(maxh=2))

In [ ]:
Draw(mesh)

In [ ]:
print(mesh.GetBoundaries())

In [ ]:
print(mesh.GetMaterials())

In [ ]:
from collections import defaultdict
from ngsolve import CoefficientFunction

In [ ]:
diel_perm = defaultdict(lambda: -1) #-1 ist default
# all diel_perm["unknwn"] = -1
diel_perm["air"] = 1.0
diel_perm_cf = CoefficientFunction([diel_perm[mat] for mat in mesh.GetMaterials()])

voltage = 4
potential = defaultdict(lambda : 0)
potential["bc_el1"] = voltage/2
potential["bc_el2"] = -voltage/2
potential_cf = CoefficientFunction([potential[bnd] for bnd in mesh.GetBoundaries()])
potential_str="bc_el1|bc_el2"

In [ ]:
Draw(potential_cf,mesh,"Potential")

In [ ]:
dirichlet_str="air|bc_el1|bc_el2"

In [ ]:
from ngsolve import H1, BilinearForm,grad,dx, GridFunction, Grad

In [ ]:
fes = H1(mesh, order = 3, dirichlet = dirichlet_str)
ut,vt = fes.TnT()
a = BilinearForm(fes)

a+= diel_perm_cf * grad(ut)*grad(vt)*dx
a.Assemble()
u = GridFunction(fes, name="Potential")
u.Set(potential_cf, definedon=mesh.Boundaries(potential_str))
f = u.vec.CreateVector()
f.data = a.mat*u.vec
u.vec.data -= a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky") *f

In [ ]:
Draw(u)

In [ ]:
# E = -Grad(u)
# Draw(E, mesh, "E")

# 3d

In [ ]:
from ngsolve.webgui import Draw
from ngsolve import Mesh
from netgen.csg import *

In [ ]:
cube1 = OrthoBrick(Pnt(-10,10,-9.5), Pnt(10,10,9.5))
air = Sphere((0,0,0),20) - cube1

geo = CSGeometry()
geo.Add(air)
geo.Add(cube1)

In [ ]:
mesh = Mesh(geo.GenerateMesh(maxh=5))
Draw (mesh)